**Imports**

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

**Dataset Preparation**

In [2]:
# loading the brest cancer dataset
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
n_samples, n_features = X.shape
print(n_samples, n_features)

# train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# applying scaling on train and test data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# converting everythin from numpy to tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(y_train)

# reshaping 
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(y_train)

569 30
torch.Size([455, 30]) torch.Size([455]) torch.Size([114, 30]) torch.Size([114])
tensor([0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
        1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
        1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
        0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
        0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
        0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 1., 0., 1., 1., 0

**Model**

In [3]:
class LogisticRegression(nn.Module):
  def __init__(self, num_inputs):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(num_inputs, 1)

  def forward(self, x): 
    y_pred = torch.sigmoid(self.linear(x))
    return(y_pred)

model = LogisticRegression(n_features)   

**Loss and Optimizer**

In [4]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

**Training Loop**

In [5]:
num_epochs = 1000
for epoch in range(num_epochs):
  # forward pass and loss
  y_pred = model(X_train)
  loss = criterion(y_pred, y_train)

  # backward pass and gradient computation 
  loss.backward()

  # update of weights
  optimizer.step()

  optimizer.zero_grad()  # making the gradients 0 

  if (epoch+1)%50==0:
    print(f'epoch = {epoch+1}, loss = {loss.item()}')

epoch = 50, loss = 0.3184701204299927
epoch = 100, loss = 0.24071693420410156
epoch = 150, loss = 0.20385263860225677
epoch = 200, loss = 0.18142607808113098
epoch = 250, loss = 0.1660081148147583
epoch = 300, loss = 0.15461508929729462
epoch = 350, loss = 0.14578238129615784
epoch = 400, loss = 0.1386934071779251
epoch = 450, loss = 0.1328517347574234
epoch = 500, loss = 0.12793640792369843
epoch = 550, loss = 0.12373003363609314
epoch = 600, loss = 0.12007958441972733
epoch = 650, loss = 0.11687420308589935
epoch = 700, loss = 0.11403131484985352
epoch = 750, loss = 0.1114882230758667
epoch = 800, loss = 0.10919621586799622
epoch = 850, loss = 0.10711692273616791
epoch = 900, loss = 0.10521969944238663
epoch = 950, loss = 0.10347964614629745
epoch = 1000, loss = 0.10187637805938721


**Model Evaluation**

In [6]:
with torch.no_grad():  # now we don't want to trach the history
  y_pred = model(X_test)
  y_pred_class = y_pred.round()
  acc = y_pred_class.eq(y_test).sum() / float(y_test.shape[0])
  print("Accuracy on the test set equals : ", acc.item())

Accuracy on the test set equals :  0.9649122953414917
